In [ ]:
from elasticsearch import Elasticsearch, helpers
import os

# Connect to Elasticsearch
ES_HOST = "http://localhost:9200"
es = Elasticsearch([ES_HOST])

In [45]:
# Delete the index if it exists
if es.indices.exists(index='index_recipes'):
    es.indices.delete(index='index_recipes')

In [ ]:
# Define the mapping for the index
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "content": {"type": "text", 
                        "analyzer": "standard",
                        "search_analyzer": "standard"}
        }
    }
}

es.indices.create(index='index_recipes', body=mapping)
print("Index 'index_recipes' with mapping created successfully.")

Index 'index_recipes' with mapping created successfully.


In [51]:
def bulk_index(directory, index_name):
    actions = []

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            path = os.path.join(directory, filename)
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()

            action = {
                "_index": index_name,
                "_source": {
                    "title": filename,
                    "content": text
                }
            }
            actions.append(action)

    # Bulk index the documents with only one call to Elasticsearch
    if actions:
        helpers.bulk(es, actions)
        print(f"{len(actions)} documents indexed in the '{index_name}' index.")
    else:
        print("No .txt files found in the directory.")

In [52]:
bulk_index("files", "index_recipes")

100 documents indexed in the 'index_recipes' index.


In [53]:
def search(query):
   res = es.search(index='index_recipes', body=query)
   print("Search Results:")
   for i, hit in enumerate(res['hits']['hits']):
    res_doc = hit['_source']
    print(f"{i+1}. Title: {res_doc['title']}\n   Content: {res_doc['content']}\n")

search({"query": {"match": {"content": "uova"}}})

Search Results:
1. Title: Crostino con uova alla benedict e pancetta.txt
   Content: Per preparare il crostino con uova alla benedict e pancetta, iniziate dalla salsa olandese: prendete il burro chiarificato, scioglietelo in un pentolino o nel microonde e tenetelo da parte. Con uno coltellino a lama liscia tritate finemente lo scalogno In un pentolino versate l’aceto l’acqua, il pepe in grani e lo scalogno e fate bollire il composto Filtrate il liquido fatto bollire, trasferendolo in una casseruola a bagnomaria (oppure predisponete un bagnomaria classico, se non si possiede una casseruola con il bagnomaria incorporato) Aggiungete anche i tuorli e muniti di frusta sbattete il composto avendo cura di continuare a sbattere costantemente, finché tutto il burro non sarà incorporato e la salsa risulterà “montata” (questa operazione richiederà più o meno 6 minuti).A questo punto togliete la casseruola dal fuoco e aggiungete il succo di limone Mettete da parte la salsa mantenendola in caldo, o